In [1]:
import pytraj as pt
import nglview as nv

_ColormakerRegistry()

In [2]:
import numpy as np
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
traj_healthy = pt.load('../ciIP/step7.1b_production.dcd', '../ciIP/step5_assembly.psf')
traj_mutant = pt.load('../Mutant/step7.1b_production.dcd', '../Mutant/step5_assembly.psf')

In [4]:
traj_healthy.xyz.shape, traj_mutant.xyz.shape

((187, 267821, 3), (363, 251836, 3))

In [5]:
# nv.show_pytraj(traj)

In [6]:
from collections import defaultdict

def residue_positions(traj):
    residue_positions = defaultdict(list)

    for index, residue in enumerate(list(traj.top.residues)):
        key = residue.name + str(residue.index)
        residue_positions[key].append(str(index + 1))
        
    return residue_positions

residue_positions_healthy = residue_positions(traj_healthy)
residue_positions_mutant = residue_positions(traj_mutant)

In [7]:
# Cal restar 1 i posar en majúscules
# Per exemple Gly209 -> GLY208
# A més, cal canviar His -> HSD

interesting_residues = [
    'ARG377', 'SER381', 'THR380', 'ARG301', 'SER206', 'GLY208',
    'LYS209', 'GLU210', 'GLY452', 'LEU453', 'PHE454', 'ILE455',
    'TYR557', 'GLY168', 'ILE169', 'GLU267', 'GLY166', 'SER167',
    'ALA164', 'HSD560', 'ARG346', 'ASN564', 'TRP342', 'ARG344',
    'LYS345', 'LYS353', 'LYS448', 'TYR566', 'LYS158', 'LYS173',
    'PHE254', 'GLY259', 'PHE263', 'GLU266', 'SER519', 'THR517',
    'LEU527', 'ALA256', 'ILE258', 'GLY260', 'LEU262', 'LEU531',
    'MET534', 'LEU537', 'MET538', 'VAL522'
]

interesting_common_residues = [r for r in interesting_residues if r not in ['LEU527', 'LEU531', 'MET534', 'VAL522']]

def get_indices(residue_positions, interesting_residues):
    s = ':'
    for index, residue in enumerate(interesting_residues):
        if len(residue_positions[residue]) == 0:
            print(f'Residue {residue} not found in traj')
        else:
            if index != 0: s += ','
            for index2, position in enumerate(residue_positions[residue]):
                if index2 != 0: s += ','
                s += position
    return s[:-1]

get_indices(residue_positions_mutant, interesting_common_residues)

':320,995,324,999,323,998,244,919,149,824,151,826,152,827,153,828,395,1070,396,1071,397,1072,398,1073,499,1174,111,786,112,787,210,885,109,784,110,785,107,782,502,1177,289,964,506,1181,285,960,287,962,288,963,296,971,391,1066,508,1183,101,776,116,791,197,872,202,877,206,881,209,884,462,1137,460,1135,199,874,201,876,203,878,205,880,479,1154,480,115'

In [8]:
get_indices(residue_positions_healthy, interesting_common_residues)

':320,996,324,1000,323,999,244,920,149,825,151,827,152,828,153,829,395,1071,396,1072,397,1073,398,1074,500,1176,111,787,112,788,210,886,109,785,110,786,107,783,503,1179,289,965,507,1183,285,961,287,963,288,964,296,972,391,1067,509,1185,101,777,116,792,197,873,202,878,206,882,209,885,462,1138,460,1136,199,875,201,877,203,879,205,881,480,1156,481,115'

In [9]:
# list(traj[get_indices(interesting_residues)].top.residues)

## Root-mean-square deviation of atomic positions

In [10]:
data_healthy_ca = traj_healthy[get_indices(residue_positions_healthy, interesting_common_residues)]['@CA'].xyz
n_frames = data_healthy_ca.shape[0]
data_healthy_ca.shape

(187, 84, 3)

In [11]:
data_mutant_ca = traj_mutant[get_indices(residue_positions_mutant, interesting_common_residues)]['@CA'].xyz[:n_frames, :, :]
data_mutant_ca.shape

(187, 84, 3)

In [12]:
def rmsd(frame1, frame2):
    dist = 0
    n_atoms = frame1.shape[0]
    for i in range(n_atoms):
        dist += np.linalg.norm(frame1[i, :] - frame2[i, :])
    return dist / n_atoms

In [13]:
def build_kernel_matrix(M):
    n_frames, n_atoms = M.shape[0], M.shape[1]
    K = np.empty(shape=(n_frames, n_frames))
    for i in range(n_frames):
        for j in range(i, n_frames):
            K[i, j] = rmsd(M[i, :, :], M[j, :, :])
            K[j, i] = K[i, j]
    
    # Obtain similarity instead of distance
    def convert_to_similarity(v):
        return (K.max() - v) / K.max()

    return K.max(), convert_to_similarity(K)

In [14]:
max_healthy, K_healthy = build_kernel_matrix(data_healthy_ca)
max_mutant, K_mutant = build_kernel_matrix(data_mutant_ca)

In [15]:
df = pd.DataFrame(K_healthy)
df = df.reset_index().rename(columns={'index': 'frame1'})
df.head()

,frame1,0,1,2,3,4,5,6,7,8,...,177,178,179,180,181,182,183,184,185,186
0,0,1.000000,0.670235,0.588344,0.612694,0.587308,0.586661,0.547028,0.573623,0.600969,...,0.216071,0.158588,0.152726,0.236379,0.198993,0.182472,0.151297,0.173246,0.190620,0.149537
1,1,0.670235,1.000000,0.652239,0.671642,0.595620,0.635151,0.541720,0.603248,0.593005,...,0.271173,0.199804,0.169632,0.281439,0.235212,0.219649,0.201996,0.227791,0.232979,0.188835
2,2,0.588344,0.652239,1.000000,0.660012,0.562698,0.606295,0.518703,0.553104,0.576137,...,0.145726,0.062613,0.034015,0.146924,0.097085,0.081559,0.058950,0.096148,0.108320,0.068755
3,3,0.612694,0.671642,0.660012,1.000000,0.625828,0.663714,0.613405,0.625329,0.632387,...,0.240555,0.176652,0.151937,0.259012,0.196157,0.173599,0.180371,0.180462,0.212250,0.186769
4,4,0.587308,0.595620,0.562698,0.625828,1.000000,0.672586,0.642173,0.691494,0.666490,...,0.239859,0.181265,0.164943,0.235134,0.174417,0.173036,0.150711,0.199550,0.233059,0.215672


In [16]:
long_form = pd.melt(df, id_vars='frame1', var_name='frame2')
long_form.head()

,frame1,frame2,value
0,0,0,1.000000
1,1,0,0.670235
2,2,0,0.588344
3,3,0,0.612694
4,4,0,0.587308


In [17]:
alt.Chart(long_form).mark_rect().encode(
    x=alt.X('frame1:O', title='frame'),
    y=alt.Y('frame2:O', title='frame'),
    color=alt.Color('value:Q', scale=alt.Scale(scheme='viridis'))
).properties(
    width=500,
    height=500
)

alt.Chart(...)

In [18]:
from sklearn.decomposition import KernelPCA
from scipy.linalg import eigh

In [ ]:
# KPCA = KernelPCA(n_components=2, kernel='precomputed')
# results = KPCA.fit_transform(K)

In [19]:
n_components = 2

def kpca(K, n_components=2):
    # Center the kernel matrix.
    N = K.shape[0]
    one_n = np.ones((N,N)) / N
    K = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)

    # Obtaining eigenpairs from the centered kernel matrix
    # numpy.eigh returns them in sorted order
    eigvals, eigvecs = eigh(K)

    # Collect the top k eigenvectors (projected samples)
    X_pc = np.column_stack((eigvecs[:, -i]
                            for i in range(1, n_components + 1)))
    
    return X_pc
    
kpca(K_mutant)

/home/victoradell/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  from ipykernel import kernelapp as app


array([[-0.11146257, -0.11412754],
       [-0.11576039, -0.13341381],
       [-0.11395621, -0.13338134],
       ...,
       [ 0.05061443, -0.04326289],
       [ 0.04333454, -0.0640287 ],
       [ 0.04415644, -0.06596937]])

In [20]:
pca_results = pd.DataFrame(kpca(K_mutant)).reset_index().rename(columns={'index': 'frame', 0: 'x', 1: 'y'})
pca_results.head()

/home/victoradell/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  from ipykernel import kernelapp as app


,frame,x,y
0,0,-0.111463,-0.114128
1,1,-0.115760,-0.133414
2,2,-0.113956,-0.133381
3,3,-0.117524,-0.123505
4,4,-0.123617,-0.112254


In [21]:
alt.Chart(pca_results).mark_point().encode(
    x=alt.X('x:Q', title='1st PC'),
    y=alt.Y('y:Q', title='2nd PC'),
    color=alt.Color('frame:Q', scale=alt.Scale(scheme='viridis')),
    tooltip='frame'
).properties(
    width=500,
    height=500
)

alt.Chart(...)